## 3. Getting Started with Neural Networks

### Classifying newswires: A Multiclass Classification Example
In the newswires problem, we want to classify newswires to one of 46 mutually exclusive topics. Since the number of classes is more than 2, this is a multiclass classification problem, extending from the binary classification problem.

In [2]:
from keras.datasets import reuters

# import numpy as np
# import pandas as pd

# from keras import models
# from keras import layers

# import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
##########
# Ingestion
##########
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

2113536/2110848 [==============================] - 6s 3us/step
